In [1]:
from biological_fuzzy_logic_networks.DREAM_analysis.utils import *

/home/ubuntu/Extra_data/miniconda3/envs/biofuzznet/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def inhibitor_mapping(reverse: bool = False):
    inhibitor_mapping = {
        "iEGFR": "EGFR",
        "iMEK": "MEK12",
        "iPI3K": "PI3K",
        "iPKC": "PKC",
        "imTOR": "mTOR",
        "EFG": "None",
        "full": "None"
    }

    if reverse:
        return {v: k for k, v in inhibitor_mapping.items()}
    else:
        return inhibitor_mapping

In [3]:
data_file = "/home/ubuntu/Extra_data/Data/DREAMdata/Time_aligned_per_cell_line/CL_incl_test/BT549.csv"
time_point = 9
non_marker_cols = ["treatment", "cell_line", "time", "cellID", "fileID"]
treatment_col_name= "treatment"
sample_n_cells = 500
filter_starved_stim = True

In [4]:
prepare_cell_line_data(
    data_file,
    time_point = time_point,
    non_marker_cols = non_marker_cols,
    treatment_col_name= treatment_col_name,
    sample_n_cells = sample_n_cells,
    filter_starved_stim = filter_starved_stim,
)

/home/ubuntu/Extra_data/Code/biological_fuzzy_logic_networks/biological_fuzzy_logic_networks/DREAM_analysis/utils.py:47: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  cl_data = pd.read_csv(data_file)


['BT549']


KeyError: 'Requested level (cell_line) does not match index name (None)'

In [5]:
if isinstance(data_file, str):
    cl_data = pd.read_csv(data_file)
elif isinstance(data_file, List):
    data = []
    for file_name in data_file:
        d = pd.read_csv(file_name)
        data.append(d)
        
    cl_data = pd.concat(data)
else:
    Exception("`data_file` should be a string or list of strings")

print(cl_data["cell_line"].unique())
data_to_nodes_map = data_to_nodes_mapping()
inhibitor_map = inhibitor_mapping()

cl_data = cl_data[cl_data["time"] == time_point]

if filter_starved_stim:
    cl_data = cl_data[cl_data["treatment"] != "full"]
    cl_data = cl_data[cl_data["time"] != 0]

if sample_n_cells:
    replacement = (
        False
        if all(
            cl_data.groupby(["cell_line", "treatment", "time"]).size()
            > sample_n_cells
        )
        else True
    )
    cl_data = cl_data.groupby(["cell_line", "treatment", "time"]).sample(
        n=sample_n_cells, replace=replacement
    )

cl_data.loc[:, "inhibitor"] = [
    inhibitor_map[treatment] for treatment in cl_data[treatment_col_name]
]

cl_data = cl_data.rename(columns=data_to_nodes_map)

['BT549']


/tmp/ipykernel_157163/92338051.py:2: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  cl_data = pd.read_csv(data_file)


KeyError: 'EGF'

In [ ]:
def split_data(
    data,
    train_treatments,
    valid_treatments,
    train_cell_lines,
    valid_cell_lines,
    do_split: bool = True,
):
    
    print(data)
    treatment_split = True
    cell_line_split = True
    if train_treatments is None and valid_treatments is None:
        treatment_split = False
    if train_cell_lines is None and valid_cell_lines is None:
        cell_line_split = False

    if not do_split:
        train = data.copy()
        valid = None
    elif not treatment_split and not cell_line_split and do_split:
        train, valid = train_test_split(data)

    else:
        if not treatment_split:
            sel_train_inhibitors = list(data["inhibitor"].unique())
            sel_valid_inhibitors = list(data["inhibitor"].unique())

        else:
            print(data)
            if train_treatments is not None and valid_treatments is not None:
                sel_train_treatments = train_treatments
                sel_valid_treatments = valid_treatments
                if not isinstance(sel_train_treatments, List):
                    sel_train_treatments = [sel_train_treatments]
                if not isinstance(valid_treatments, List):
                    sel_valid_treatments = [sel_valid_treatments]

                if (
                    not len(set(sel_train_treatments).intersection(set(sel_valid_treatments)))
                    == 0
                ):
                    raise Exception("Given train and validation treatments overlap")
                else:
                    inhibitor_map = inhibitor_mapping()
                    sel_train_inhibitors = [
                        inhibitor_map[treatment] for treatment in sel_train_treatments
                    ]
                    sel_valid_inhibitors = [
                        inhibitor_map[treatment] for treatment in sel_valid_treatments
                    ]

            elif train_treatments is not None:
                sel_train_treatments = train_treatments
                print("Splitting based on train treatment(s)")
                if not isinstance(sel_train_treatments, List):
                    sel_train_treatments = [sel_train_treatments]

                inhibitor_map = inhibitor_mapping()
                sel_train_inhibitors = [
                    inhibitor_map[treatment] for treatment in sel_train_treatments
                ]
                sel_valid_inhibitors = [
                    inhibitor
                    for inhibitor in data["inhibitor"].unique()
                    if inhibitor not in sel_train_inhibitors
                ]

            elif valid_treatments is not None:
                sel_valid_treatments = valid_treatments
                print("Splitting based on validation treatment(s)")
                if not isinstance(sel_valid_treatments, List):
                    sel_valid_treatments = [sel_valid_treatments]

                inhibitor_map = inhibitor_mapping()
                sel_valid_inhibitors = [
                    inhibitor_map[treatment] for treatment in sel_valid_treatments
                ]
                sel_train_inhibitors = [
                    inhibitor
                    for inhibitor in data["inhibitor"].unique()
                    if inhibitor not in sel_valid_inhibitors
                ]

        print(data)
        if not cell_line_split:
            sel_train_cls = list(data["cell_line"].unique())
            sel_valid_cls = list(data["cell_line"].unique())

        else:
            if train_cell_lines is not None and valid_cell_lines is not None:
                sel_train_cls = train_cell_lines
                sel_valid_cls = valid_cell_lines
                if not isinstance(sel_train_cls, List):
                    sel_train_cls = [sel_train_cls]
                if not isinstance(sel_valid_cls, List):
                    sel_valid_cls = [sel_valid_cls]

                if (
                    not len(set(sel_train_cls).intersection(set(sel_valid_cls)))
                    == 0
                ):
                    raise Exception("Given train and validation cell lines overlap")

            elif train_cell_lines is not None:
                print("Splitting based on train cell line(s)")
                sel_train_cls = train_cell_lines
                if not isinstance(sel_train_cls, List):
                    sel_train_cls = [sel_train_cls]

                sel_valid_cls = [
                    cl
                    for cl in data["cell_line"].unique()
                    if cl not in sel_train_cls
                ]

            elif valid_cell_lines is not None:
                print("Splitting based on validation cell line(s)")
                sel_valid_cls = valid_cell_lines
                if not isinstance(sel_valid_cls, List):
                    sel_valid_cls = [sel_valid_cls]

                sel_train_cls = [
                    cl
                    for cl in data["cell_line"].unique()
                    if cl not in sel_valid_cls
                ]
    
        train = data.loc[
            (data["cell_line"].isin(sel_train_cls))
            & data["inhibitor"].isin(sel_train_inhibitors),
            :,
        ]
        valid = data.loc[
            (data["cell_line"].isin(sel_valid_cls))
            & data["inhibitor"].isin(sel_valid_inhibitors),
            :,
        ]

    print(sel_valid_inhibitors)
    print(sel_valid_cls)
    print(sel_train_inhibitors)
    print(sel_train_cls)
    return train, valid

In [6]:
model = create_bfz("/home/ubuntu/Extra_data/Data/DREAMdata/PKN_Alice.sif", "DREAMBioFuzzNet")

In [7]:
data = cl_data.copy()
model
train_treatments = ["iEGFR", "iPI3K", "iMEK"]
valid_treatments = "iPKC"
test_treatments = "imTOR"
train_cell_lines = None
valid_cell_lines = None
inhibition_value = 1.0
minmaxscale = True
add_root_values = True
input_value = 1
root_nodes = ["EGF", "SERUM"]
do_split = True
replace_zero_inputs = 1e-9


In [8]:
markers = [c for c in data.columns if c in model.nodes()]
if isinstance(replace_zero_inputs, float):
    do_replace = True
    replace_value = replace_zero_inputs
elif replace_zero_inputs:
    do_replace = True
    replace_value = 1e-9
else:
    do_replace = False

data = data.dropna(subset=markers, axis=0)
print("Data:", data)

train, valid = split_data(
    data,
    train_treatments,
    valid_treatments,
    train_cell_lines,
    valid_cell_lines,
    do_split=do_split,
)
if isinstance(minmaxscale, bool):
    if minmaxscale:
        scaler = MinMaxScaler()
        scaler.fit(train[markers])
        train[markers] = scaler.transform(train[markers])
        if valid is not None:
            valid[markers] = scaler.transform(valid[markers])
            t = valid[markers]
            t[t < 0] = 0
            valid[markers] = t
    else:
        scaler = None
elif minmaxscale is not None:
    scaler = minmaxscale
    train[markers] = scaler.transform(train[markers])
    t = train[markers]
    t[t < 0] = 0
    train[markers] = t
    if valid is not None:
        valid[markers] = scaler.transform(valid[markers])
        t = valid[markers]
        t[t < 0] = 0
        valid[markers] = t
else:
    scaler = None
if add_root_values:
    train.loc[:, root_nodes] = input_value
    if valid is not None:
        valid.loc[:, root_nodes] = input_value

if do_replace:
    for node in model.root_nodes:
        train.loc[train[node] == 0, node] = replace_value

train_dict = train.to_dict("list")
train_data = {
    k: DoubleTensor(v) for k, v in train_dict.items() if k in model.nodes()
}
train_inhibitors = {
    m1: DoubleTensor(
        [inhibition_value if m == m1 else 1.0 for m in train_dict["inhibitor"]]
    )
    for m1 in model.nodes()
}
train_input = {node: train_data[node] for node in model.root_nodes}

if valid is not None:
    if do_replace:
        for node in model.root_nodes:
            valid.loc[valid[node] == 0, node] = replace_value
    valid_dict = valid.to_dict("list")

    valid_data = {
        k: DoubleTensor(v) for k, v in valid_dict.items() if k in model.nodes()
    }

    valid_inhibitors = {
        m1: DoubleTensor(
            [inhibition_value if m == m1 else 1.0 for m in valid_dict["inhibitor"]]
        )
        for m1 in model.nodes()
    }
    valid_input = {node: valid_data[node] for node in model.root_nodes}

Data:        treatment cell_line  time  cellID  fileID  b.CATENIN  cleavedCas  \
18707        EGF     BT549   9.0  5502.0  3480.0   0.224784    2.481006   
136047       EGF     BT549   9.0  2960.0  3547.0   1.572994    3.902246   
16611        EGF     BT549   9.0  3406.0  3480.0   0.224784    0.988647   
136390       EGF     BT549   9.0  3303.0  3547.0   1.177029    2.424062   
139321       EGF     BT549   9.0  6234.0  3547.0   0.542027    1.690555   
...          ...       ...   ...     ...     ...        ...         ...   
310683     imTOR     BT549   9.0   672.0  3489.0   0.782945    2.362751   
312652     imTOR     BT549   9.0  2641.0  3489.0   0.879023    1.091342   
310067     imTOR     BT549   9.0    56.0  3489.0   0.328161    1.066391   
314621     imTOR     BT549   9.0  4610.0  3489.0   1.276375    3.160479   
310033     imTOR     BT549   9.0    22.0  3489.0   0.908289    0.898342   

        CyclinB     GAPDH       IdU  ...  p.PLCg2     p.RB     p.S6     p.S6K  \
18707   5.62

KeyError: 'inhibitor'

In [12]:
cl_data.isna().sum()

treatment           0
cell_line           0
time                0
cellID              0
fileID              0
b.CATENIN           0
cleavedCas          0
CyclinB             0
GAPDH               0
IdU                 0
Ki.67               0
p.4EBP1             0
p.Akt.Ser473.       0
p.AKT.Thr308.       0
p.AMPK              0
p.BTK               0
p.CREB              0
p.ERK               0
p.FAK               0
p.GSK3b             0
p.H3                0
p.HER2            500
p.JNK               0
p.MAP2K3            0
p.MAPKAPK2          0
p.MEK               0
p.MKK3.MKK6         0
p.MKK4              0
p.NFkB              0
p.p38               0
p.p53               0
p.p90RSK            0
p.PDPK1             0
p.PLCg2          3000
p.RB                0
p.S6                0
p.S6K               0
p.SMAD23            0
p.SRC               0
p.STAT1             0
p.STAT3             0
p.STAT5             0
time_course       500
dtype: int64

In [ ]:
train = data.loc[
            (data["cell_line"].isin(train_cell_lines))
            & data["inhibitor"].isin(train_inhibitors),
            :,
        ]

In [ ]:
train_cell_lines

In [ ]:
list(cl_data["cell_line"].unique())

In [ ]:
pd.Series(["None", "None", "A"]).dropna()